# Graph Neural Network for Stress Prediction on FEA Mesh Data
This notebook trains a Graph Convolutional Network (GCN) to predict **von Mises stress** from Finite Element Analysis (FEA) mesh data, using randomized Young’s modulus scalar fields `E` as input features.  
The model is trained on GPU (if available) for faster training and evaluation.

In [40]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

try:
    import torch_geometric
    from torch_geometric.data import Data, Dataset, InMemoryDataset, DataLoader as GeoDataLoader
    from torch_geometric.loader import DataLoader
    from torch_geometric.nn import GCNConv
    GEOMETRIC_AVAILABLE = True
except Exception as e:
    GEOMETRIC_AVAILABLE = False
    print('torch_geometric not available. GNN-related cells will show instructions to install it.')
    raise ImportError("torch_geometric is required to run this notebook.")

## Config
Set up dataset paths, split ratios, batch size, training parameters, and device.


In [ ]:
PROCESSED_DIR = "../data/DataSet1000/processed_data"
MESH_META_PATH = os.path.join(PROCESSED_DIR, "mesh_metadata.npz")
SPLIT_RATIOS = (0.7, 0.15, 0.15)
BATCH_SIZE = 8
EPOCHS = 10
LR = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Single Graph Function
Defines a helper to load one graph sample (features, edges, and target) from disk.


In [33]:
def load_graph(sample_path, mesh_meta_path):
    meta = np.load(mesh_meta_path)
    coords = meta["coords"].astype(np.float32)
    edge_index = meta["edge_index"].astype(np.int64)
    arr = np.load(sample_path)
    E = arr["E"].astype(np.float32)
    y = arr["y"].astype(np.float32)

    x = np.concatenate([coords, E.reshape(-1, 1)], axis=1).astype(np.float32)

    x = torch.from_numpy(x)
    y = torch.from_numpy(y.reshape(-1, 1))

    return Data(
        x=x,
        edge_index=torch.from_numpy(edge_index),
        y=y
    )

## MeshDataset Class
Custom dataset class for loading mesh graph samples and applying normalization.


In [24]:
class MeshDataset(Dataset):
    def __init__(self, files, mesh_meta_path, x_mean=None, x_std=None, y_mean=None, y_std=None):
        super().__init__()
        self.files = files
        self.mesh_meta_path = mesh_meta_path
        self.x_mean = x_mean
        self.x_std = x_std
        self.y_mean = y_mean
        self.y_std = y_std

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        data = load_graph(self.files[idx], self.mesh_meta_path)

        if self.x_mean is not None and self.x_std is not None:
            data.x = (data.x - self.x_mean) / self.x_std.clamp(min=1e-8)
        if self.y_mean is not None and self.y_std is not None:
            data.y = (data.y - self.y_mean) / self.y_std.clamp(min=1e-8)

        return data


## Prepare Data
Collect all sample files, shuffle them, and split into train/val/test sets.


In [25]:
sample_files = sorted(
    f for f in os.listdir(PROCESSED_DIR)
    if f.startswith("sample_") and f.endswith(".npz")
)
sample_files = [os.path.join(PROCESSED_DIR, f) for f in sample_files]

import random
random.seed(42)
random.shuffle(sample_files)

n_total = len(sample_files)
n_train = int(n_total * SPLIT_RATIOS[0])
n_val = int(n_total * SPLIT_RATIOS[1])

train_files = sample_files[:n_train]
val_files = sample_files[n_train:n_train + n_val]
test_files = sample_files[n_train + n_val:]

print(f"Samples total: {n_total} | train: {len(train_files)} | val: {len(val_files)} | test: {len(test_files)}")


Samples total: 1000 | train: 700 | val: 150 | test: 150


## Compute Normalization Parameters
Calculate feature and target mean/std using training data for normalization.


In [34]:
all_x = []
all_y = []

if len(train_files) == 0:
    raise RuntimeError("Train file list is empty. Cannot compute normalization stats.")

for file in train_files:
    data = load_graph(file, MESH_META_PATH)
    if data.x.numel() == 0 or data.y.numel() == 0:
        continue  # to skip empty data
    all_x.append(data.x)
    all_y.append(data.y)

if len(all_x) == 0 or len(all_y) == 0:
    raise RuntimeError("No valid training data found to compute normalization parameters.")

all_x = torch.cat(all_x, dim=0)
all_y = torch.cat(all_y, dim=0)

x_mean = torch.mean(all_x, dim=0)
x_std = torch.std(all_x, dim=0)
y_mean = torch.mean(all_y, dim=0)
y_std = torch.std(all_y, dim=0)

print("Feature Mean:", x_mean)
print("Feature Std Dev:", x_std)
print("Target Mean:", y_mean)
print("Target Std Dev:", y_std)

normalization_params = {
    'x_mean': x_mean,
    'x_std': x_std,
    'y_mean': y_mean,
    'y_std': y_std
}

torch.save(normalization_params, "normalization_params.pth")
print("Normalization parameters saved to normalization_params.pth")


Feature Mean: tensor([-3.0867e-03, -2.0257e-04,  9.9990e-02,  2.4715e+01])
Feature Std Dev: tensor([ 0.7335,  0.7335,  0.0773, 12.4010])
Target Mean: tensor([367930.9688])
Target Std Dev: tensor([14710574.])
Normalization parameters saved to normalization_params.pth


## Create Datasets and DataLoaders
Instantiate training, validation, and test datasets and wrap them with loaders.


In [35]:
train_dataset = MeshDataset(train_files, MESH_META_PATH, x_mean, x_std, y_mean, y_std)
val_dataset = MeshDataset(val_files, MESH_META_PATH, x_mean, x_std, y_mean, y_std)
test_dataset = MeshDataset(test_files, MESH_META_PATH, x_mean, x_std, y_mean, y_std)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


## GCN Model
Define the Graph Convolutional Network (GCN) architecture for regression.


In [28]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.relu = torch.nn.ReLU()
        self.conv2 = GCNConv(hidden_channels, 128)
        self.conv3 = GCNConv(128, 64)
        self.fc1 = torch.nn.Linear(64, 256)
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc3 = torch.nn.Linear(128, 64)
        self.fc4 = torch.nn.Linear(64, 32)
        self.fc5 = torch.nn.Linear(32, out_channels)

    def forward(self, x, edge_index):
        x = self.relu(self.conv1(x, edge_index))
        x = self.relu(self.conv2(x, edge_index))
        x = self.relu(self.conv3(x, edge_index))
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x

def build_gnn(in_channels, hidden_channels, out_channels):
    return GCN(in_channels, hidden_channels, out_channels)


## Training Function
Defines one training epoch: forward pass, loss computation, and backpropagation.


In [36]:
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(DEVICE)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = loss_fn(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


## Evaluation Function
Evaluates the model on a dataset, returning MAE, MSE, and R² scores.


In [38]:
def evaluate(model, loader):
    model.eval()
    y_true, y_pred = [], []
    normalization_params = torch.load("normalization_params.pth")
    y_mean = normalization_params['y_mean']
    y_std = normalization_params['y_std']

    with torch.no_grad():
        for batch in loader:
            batch = batch.to(DEVICE)
            pred = model(batch.x, batch.edge_index)

            pred_cpu = pred.cpu()
            true_cpu = batch.y.cpu()

            denorm_pred = pred_cpu * y_std + y_mean
            denorm_true = true_cpu * y_std + y_mean

            y_pred.append(denorm_pred.numpy().flatten())
            y_true.append(denorm_true.numpy().flatten())

    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)

    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "R2": r2_score(y_true, y_pred)
    }


## Run Training and Validation
Train the model for multiple epochs and evaluate on validation set.


In [ ]:
sample_input_dim = train_dataset[0].x.shape[1]
model = build_gnn(sample_input_dim, 128, 1).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = torch.nn.MSELoss()

print(f"Starting training for {EPOCHS} epochs...")
for epoch in range(EPOCHS):
    loss = train(model, train_loader, optimizer, loss_fn)
    print(f"Epoch {epoch+1}/{EPOCHS} - Train MSE: {loss:.6f}")

metrics = evaluate(model, val_loader)
print("\nValidation:", metrics)

torch.save(model.state_dict(), "gnn_model_normalized.pth")
print("✅ Model saved")